# Evaluating tfcoder completions

In [1]:
import os
import sys
from pathlib import Path

CURRENT_DIRECTORY = Path(os.getcwd())
ROOT_DIRECTORY = (CURRENT_DIRECTORY / "..").absolute().resolve()

print(f"Current directory: {CURRENT_DIRECTORY}")
print(f"Root directory: {ROOT_DIRECTORY}")

sys.path.append(str(ROOT_DIRECTORY))

Current directory: /home/ubuntu/arga-arc/tf_coder
Root directory: /home/ubuntu/arga-arc


In [2]:
import typing as t
import json
from pprint import pprint
from dataclasses import dataclass
import tensorflow as tf
import numpy as np
import math

2024-02-23 18:36:39.515117: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-23 18:36:39.518366: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-23 18:36:39.557932: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-23 18:36:39.557958: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-23 18:36:39.558803: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [3]:
class OutputJSON(t.TypedDict):
    task_id: str
    completions: t.List[str]
    coverage_percentage: float
    description: str
    tf_operators: t.Dict[str, int]
    total_covered: int
    total_in_target: int

class ExamplesJSON(t.TypedDict):
    inputs: str
    outputs: str

class TaskJSON(t.TypedDict):
    constants: str
    description: str
    name: str
    source: str
    target_program: str
    examples: ExamplesJSON



OUTPUT_FILE = CURRENT_DIRECTORY / "output_tfcoder.json"
DATASET_FILE = CURRENT_DIRECTORY / "tfcoder_dataset.json"

OUTPUT: t.List[OutputJSON] = json.loads(OUTPUT_FILE.read_text())
DATASET: t.List[TaskJSON] = json.loads(DATASET_FILE.read_text())

print(f"Loaded {len(OUTPUT)} outputs and {len(DATASET)} tasks")

TASK_JSONS = {task["name"]: task for task in DATASET}
OUTPUTS = {output["task_id"]: output for output in OUTPUT}

keys = list(TASK_JSONS.keys())[0:5]
for key in keys:
    print(f"Task: {key}")
    pprint(TASK_JSONS[key])
    pprint(OUTPUTS[key])
    print()

Loaded 72 outputs and 72 tasks
Task: google_01
{'constants': '[]',
 'description': 'Convert index tensor into pairs for SparseTensor indexing',
 'examples': {'inputs': '[[0, 0, 0, 1, 3, 3],]',
              'outputs': '[[0, 0], [0, 1], [0, 2], [1, 0], [3, 0], [3, 1]]'},
 'name': 'google_01',
 'source': 'From an internal Google chat room, 09/07/2018',
 'target_program': 'tf.cast(tf.where(tf.sequence_mask(tf.math.bincount(in1))), '
                   'tf.int32)'}
{'completions': ['\n'
                 'def convert_to_pairs(tensor):\n'
                 '    return tf.reshape(tensor, (-1, 2))\n'
                 '\n'
                 'tensor = tf.constant([0, 0, 0, 1, 3, 3])\n'
                 'pairs = convert_to_pairs(tensor)\n',
                 '\n'
                 'def convert_indices(indices):\n'
                 '    indices = tf.constant(indices)\n'
                 '    second_elements = tf.where(indices)\n'
                 '    first_elements = tf.subtract(second_elements, 1)\n

In [4]:
@dataclass
class Example:
    inputs: t.List[np.ndarray]
    output: np.ndarray

    @classmethod
    def from_json(cls, examples: ExamplesJSON):
        try:
            evaluated_inputs = eval(examples["inputs"])
            if isinstance(evaluated_inputs, list):
                inputs = [np.array(i) for i in evaluated_inputs]
            else:
                inputs = [evaluated_inputs]
        except Exception as e:
            print(f"Error evaluating inputs: {e}")
            print(f"Inputs: {examples['inputs']}")
            raise e

        try:
            evaluated_outputs = eval(examples["outputs"])
            if isinstance(evaluated_outputs, list):
                outputs = np.array(evaluated_outputs)
            if isinstance(evaluated_outputs, tf.Tensor):
                outputs = evaluated_outputs.numpy()
            else:
                outputs = evaluated_outputs
        except Exception as e:
            print(f"Error evaluating outputs: {e}")
            print(f"Outputs: {examples['outputs']}")
            raise e

        return cls(inputs, outputs)

SKIP_TASKS = [] # ["google_13", "stackoverflow_05", "stackoverflow_10", "stackoverflow_13"]

# for each task in dataset, show inputs and outputs
for task in DATASET:
    if task["name"] in SKIP_TASKS:
        continue
    print(f"Task name: {task['name']}")
    pprint(Example.from_json(task["examples"]))
    print()

Task name: google_01
Example(inputs=[array([0, 0, 0, 1, 3, 3])],
        output=array([[0, 0],
       [0, 1],
       [0, 2],
       [1, 0],
       [3, 0],
       [3, 1]]))

Task name: google_02
Example(inputs=[array([[0., 1., 0., 0.],
       [0., 1., 1., 0.],
       [1., 1., 1., 1.]])],
        output=array([[0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.5 , 0.5 , 0.  ],
       [0.25, 0.25, 0.25, 0.25]]))

Task name: google_03
Example(inputs=[SparseTensor(indices=tf.Tensor(
[[0 0 0]
 [0 1 1]
 [1 1 1]
 [1 1 2]], shape=(4, 3), dtype=int64), values=tf.Tensor([1. 1. 1. 1.], shape=(4,), dtype=float32), dense_shape=tf.Tensor([  2   2 800], shape=(3,), dtype=int64))],
        output=SparseTensor(indices=tf.Tensor(
[[0 0 0]
 [0 1 1]], shape=(2, 3), dtype=int64), values=tf.Tensor([1. 1.], shape=(2,), dtype=float32), dense_shape=tf.Tensor([  1   2 800], shape=(3,), dtype=int64)))

Task name: google_04
Example(inputs=[array([111, 112, 121, 122, 131, 132, 211, 212, 221, 222, 231, 232, 311,
       312, 

2024-02-23 18:36:41.120520: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
@dataclass
class Task:
    name: str
    description: str
    target_program: str
    examples: Example
    completions: t.List[str]

    @classmethod
    def from_json(cls, task: TaskJSON, output: OutputJSON):
        assert task["name"] == output["task_id"]
        return cls(
            task["name"],
            task["description"],
            task["target_program"],
            Example.from_json(task["examples"]),
            output["completions"]
        )

TASKS = {
    task["name"]: Task.from_json(task, OUTPUTS[task["name"]])
    for task in DATASET
}

# pprint 5 tasks
keys = list(TASKS.keys())[0:5]
for key in keys:
    print(f"Task: {key}")
    pprint(TASKS[key])
    print()

Task: google_01
Task(name='google_01',
     description='Convert index tensor into pairs for SparseTensor indexing',
     target_program='tf.cast(tf.where(tf.sequence_mask(tf.math.bincount(in1))), '
                    'tf.int32)',
     examples=Example(inputs=[array([0, 0, 0, 1, 3, 3])],
                      output=array([[0, 0],
       [0, 1],
       [0, 2],
       [1, 0],
       [3, 0],
       [3, 1]])),
     completions=['\n'
                  'def convert_to_pairs(tensor):\n'
                  '    return tf.reshape(tensor, (-1, 2))\n'
                  '\n'
                  'tensor = tf.constant([0, 0, 0, 1, 3, 3])\n'
                  'pairs = convert_to_pairs(tensor)\n',
                  '\n'
                  'def convert_indices(indices):\n'
                  '    indices = tf.constant(indices)\n'
                  '    second_elements = tf.where(indices)\n'
                  '    first_elements = tf.subtract(second_elements, 1)\n'
                  '    index_pairs = tf.s

In [22]:
TEST_COMPLETIONS = [
"""
input = tf.constant([[1, 2, 3, 4], [5, 6, 7, 8]])

output = tf.repeat(input, repeats=6, axis=0)
"""
]

def evaluate_completion(completion: str) -> np.ndarray:
    """
    evaluates the completion, returning any local variables with an 
    array-like value
    """
    locals_dict = {}
    exec(completion, None, locals_dict)

    return [
        value if isinstance(value, np.ndarray)
        else value.numpy() if isinstance(value, tf.Tensor)
        else np.array(value) if isinstance(value, list)
        else None
        for key, value in locals_dict.items()
        if isinstance(value, np.ndarray) or isinstance(value, tf.Tensor) or isinstance(value, list)
    ]
        

pprint([evaluate_completion(completion) for completion in TEST_COMPLETIONS])

[[array([[1, 2, 3, 4],
       [5, 6, 7, 8]], dtype=int32),
  array([[1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4],
       [5, 6, 7, 8],
       [5, 6, 7, 8],
       [5, 6, 7, 8],
       [5, 6, 7, 8],
       [5, 6, 7, 8],
       [5, 6, 7, 8]], dtype=int32)]]
